In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.utils import class_weight

In [3]:
# Constants
base_path = '/content/drive/My Drive/Dataset' # Make sure to replace 'path_to_images' with the correct path
categories = ['category1_tumor', 'category2_tumor', 'category3_tumor', 'no_tumor']

In [4]:
X = []
y = []

for idx, category in enumerate(categories):
    folder_path = os.path.join(base_path, category)
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (150, 150))
        X.append(img)
        y.append(idx)

X = np.array(X).reshape(-1, 150, 150, 1) / 255.0
y = to_categorical(y, num_classes=len(categories))



In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [6]:
# Compute class weights
class_weights =class_weight = {0:0.815, 1:0.868, 2:0.862, 3:2.17}

In [7]:
from tensorflow.keras.regularizers import L2
model1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1),padding='SAME'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(12, activation='relu'),

    Dropout(0.5),
    Dense(len(categories), activation='softmax')
])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model1.fit(X_train, y_train, epochs=40, batch_size=60, validation_data=(X_val, y_val), class_weight=class_weights)


Epoch 1/40
24/37 [==================>...........] - ETA: 43s - loss: 1.3944 - accuracy: 0.2569

In [ ]:
test_loss, test_acc = model1.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# 1. Predictions on test data
y_pred = model1.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# 2. Classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=categories))

# 3. Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# 4. Training history plots
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()
